In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta


agency = pd.read_csv('Data/INTERCITE/agency.txt')
feed_info = pd.read_csv('Data/INTERCITE/feed_info.txt')
transfers = pd.read_csv('Data/INTERCITE/transfers.txt')
calendar_dates = pd.read_csv('Data/INTERCITE/calendar_dates.txt')
routes = pd.read_csv('Data/INTERCITE/routes.txt')
stop_times = pd.read_csv('Data/INTERCITE/stop_times.txt')
stops = pd.read_csv('Data/INTERCITE/stops.txt')
trips = pd.read_csv('Data/INTERCITE/trips.txt')

In [2]:
stops.fillna('', inplace=True)
calendar_dates.date = pd.to_datetime(calendar_dates["date"], format='%Y%m%d').dt.date
stops

C:\Users\thege\AppData\Local\Temp\ipykernel_20476\221284619.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  stops.fillna('', inplace=True)


,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,StopArea:OCE87393009,Versailles Chantiers,,48.795826,2.135883,,,1,
1,StopPoint:OCEOUIGO-87393009,Versailles Chantiers,,48.795826,2.135883,,,0,StopArea:OCE87393009
2,StopArea:OCE87393579,Massy-Palaiseau,,48.726421,2.257528,,,1,
3,StopPoint:OCEOUIGO-87393579,Massy-Palaiseau,,48.726421,2.257528,,,0,StopArea:OCE87393579
4,StopArea:OCE87394007,Chartres,,48.448202,1.481313,,,1,
...,...,...,...,...,...,...,...,...,...
294,StopPoint:OCEINTERCITES-87783423,Saint-Georges-de-Luzençon,,44.066553,2.983784,,,0,StopArea:OCE87783423
295,StopArea:OCE87783431,Saint-Rome-de-Cernon,,44.013981,2.967374,,,1,
296,StopPoint:OCEINTERCITES-87783431,Saint-Rome-de-Cernon,,44.013981,2.967374,,,0,StopArea:OCE87783431
297,StopArea:OCE87783456,Tournemire - Roquefort,,43.969800,3.016626,,,1,


In [46]:
# Exemple de ville : StopArea:OCE87586545

class AnalyzerGTFS:
    stop_ids = [] #Identifiants des StopsPoints de la ville
    trip_ids = [] #Identifiants des Trajets liés à tous les StopPoints de la ville
    service_id = [] #Identifiants des services faisant des trajets à des dates comprises entre date_min et date_max
    stop_time_trip = [] # Contient StopPoints, Trajets et horaires des trajets
    trip = [] # Contient les trip_id, route_id et les services_id
    tripDates = []  # Contient les trajets des services fonctionnant sur les dates données
    stops_trip_id = [] # Contient tous les stops des trajets corrects
    date = [] # Contient tous les services et toutes les dates comprises entre date_min et date_max, (Utile pour afficher à l'utilisateur toutes les dates possibles pour le trajet)
    destinations = [] # Contient les stops après la ville (StopSequence > StopSequence de la ville)
    def __init__(self, id_ville, date_min,date_max):
        self.id_ville = id_ville
        self.date_min = datetime.strptime(date_min, '%Y%m%d').date()
        self.date_max = datetime.strptime(date_max, '%Y%m%d').date()
    
    # Etape 1 : Récupérer les stops de la ville / StopArea
    def get_stops(self):
        self.stop_ids = stops[stops['parent_station'].str.contains(self.id_ville)]['stop_id']
        return self.stop_ids
    
    # Etape 2 : Récupérer les trajets des StopPoints, peut importe la date
    def get_trips(self):
        self.stop_time_trip = pd.merge(stop_times, self.stop_ids, on='stop_id')
        self.trip_ids = self.stop_time_trip['trip_id']
        return self.trip_ids   

    # Etape 3 : Récupérer les services_id des Trajets
    def get_service_id(self):
        self.trip = pd.merge(self.trip_ids, trips, on='trip_id')
        self.service_id = self.trip['service_id']
        return self.service_id 
    
    # Etape 4 : Récupérer les services fonctionnant sur les dates données
    def get_dates(self):
        service_id = self.get_service_id()
        dates = pd.merge(service_id, calendar_dates, on='service_id')
        self.date = dates[(dates['date'] >= self.date_min) & (dates['date'] <= self.date_max)]
        return self.date
    
    # Etape 5 : Récupérer les trajets des services fonctionnant sur les dates données
    def get_trajets(self):
        services_uniques = self.date.drop_duplicates(subset='service_id')
        self.trajets = pd.merge(services_uniques, self.trip, on='service_id')
        return self.trajets
    
    # Etape 6 : Récupérer les stops des trajets corrects
    def get_stops_trajets(self):
        self.stops_trip_id = pd.merge(self.trajets, stop_times, on='trip_id')
        return self.stops_trip_id
    
    # Etape 7 : Récupérer les ids des destinations, c'est à dire les stops après la ville (StopSequence > StopSequence de la ville)
    def get_stops_destinations(self):
        Time = self.stop_time_trip['departure_time'] #StopTime trip
        Time.index = self.stop_time_trip['trip_id']

        self.stops_trip_id = self.stops_trip_id.assign(TempsVille = "")
        value=Time.loc[self.stops_trip_id['trip_id']]

        self.stops_trip_id['TempsVille'] = value.array

        self.destinations = self.stops_trip_id[self.stops_trip_id['departure_time'] > self.stops_trip_id['TempsVille']]
        return self.destinations
    
    # Etape 8 : Récupérer les destinations
    def get_destinations(self):
        destinations_uniques = self.destinations.drop_duplicates(subset='stop_id')
        return pd.merge(destinations_uniques, stops, on='stop_id')

In [50]:
villeEx = 'StopArea:OCE87763300'

Analyzer = AnalyzerGTFS(villeEx,'20240601','20240731')
Analyzer.get_stops()
Analyzer.get_trips()
Analyzer.get_service_id()
Analyzer.get_dates()
Analyzer.get_trajets()
Analyzer.get_stops_trajets()
Analyzer.get_stops_destinations()
Analyzer.get_destinations()

,service_id,date,exception_type,trip_id,route_id,trip_headsign,direction_id,block_id,shape_id,arrival_time,...,shape_dist_traveled,TempsVille,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,30:45:00,...,NaN,30:22:00,Gap,,44.563714,6.085669,,,0,StopArea:OCE87763003
1,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,31:07:00,...,NaN,30:22:00,Chorges,,44.545842,6.271889,,,0,StopArea:OCE87763425
2,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,31:42:00,...,NaN,30:22:00,Embrun,,44.567012,6.496753,,,0,StopArea:OCE87763466
3,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,32:04:00,...,NaN,30:22:00,Mont-Dauphin - Guillestre,,44.674625,6.615610,,,0,StopArea:OCE87763508
4,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,32:18:00,...,NaN,30:22:00,L'Argentière les Écrins,,44.791013,6.556408,,,0,StopArea:OCE87763557
5,13,2024-06-29,1,OCESN5787F2998294:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5787,1.0,774,NaN,32:38:00,...,NaN,30:22:00,Briançon,,44.889842,6.632966,,,0,StopArea:OCE87763607
6,13,2024-06-29,1,OCESN5792F3067053:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5792,0.0,860,NaN,23:13:00,...,NaN,22:08:00,Die,,44.758238,5.363219,,,0,StopArea:OCE87761841
7,13,2024-06-29,1,OCESN5792F3067053:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5792,0.0,860,NaN,23:50:00,...,NaN,22:08:00,Crest,,44.730196,5.016106,,,0,StopArea:OCE87761783
8,13,2024-06-29,1,OCESN5792F3067053:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5792,0.0,860,NaN,31:53:00,...,NaN,22:08:00,Paris Austerlitz,,48.842285,2.364891,,,0,StopArea:OCE87547000
9,190,2024-07-13,1,OCESN5792F3093091:2024-06-26T00:35:36Z,FR:Line::5648ced8-53ec-4327-a902-803b8aeeb1d9:,5792,0.0,873,NaN,22:54:00,...,NaN,22:08:00,Luc-en-Diois,,44.613697,5.454618,,,0,StopArea:OCE87761874


In [51]:
Destinations = Analyzer.get_destinations()

In [71]:
import folium

city = stops.loc[stops['stop_id'] == villeEx]

map = folium.Map(location=[float(city.loc[:,['stop_lat']].values[0][0]), float(city.loc[:,['stop_lon']].values[0][0])], zoom_start=7)
folium.CircleMarker([float(city.loc[:,['stop_lat']].values[0][0]), float(city.loc[:,['stop_lon']].values[0][0])], radius = 2, popup=city['stop_name'], color = "red").add_to(map)

for row in Destinations.itertuples():
    folium.CircleMarker([float(row.stop_lat), float(row.stop_lon)], radius = 2, popup=row.stop_name).add_to(map)
map

In [64]:
Time = Analyzer.stop_time_trip['departure_time']
Time.index = Analyzer.stop_time_trip['trip_id']

Analyzer.stops_trip_id = Analyzer.stops_trip_id.assign(TempsVille = "")
value=Time.loc[Analyzer.stops_trip_id['trip_id']]

Analyzer.stops_trip_id['TempsVille'] = value.array

Superieur = Analyzer.stops_trip_id[Analyzer.stops_trip_id['departure_time'] > Analyzer.stops_trip_id['TempsVille']]
Superieur

,service_id,date,exception_type,trip_id,route_id,trip_headsign,direction_id,block_id,shape_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,TempsVille
2,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,10:08:00,10:10:00,StopPoint:OCEINTERCITES-87611244,2,NaN,0,0,NaN,09:06:00
3,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,10:38:00,10:43:00,StopPoint:OCEINTERCITES-87611004,3,NaN,0,0,NaN,09:06:00
4,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,11:28:00,11:31:00,StopPoint:OCEINTERCITES-87615286,4,NaN,0,0,NaN,09:06:00
5,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,12:00:00,12:04:00,StopPoint:OCEINTERCITES-87781104,5,NaN,0,0,NaN,09:06:00
6,227,2024-07-01,1,OCESN4657F3063601:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4657,1.0,592,NaN,12:37:00,12:39:00,StopPoint:OCEINTERCITES-87773200,6,NaN,0,0,NaN,09:06:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,263,2024-07-12,1,OCESN4758F3124374:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4758,0.0,656,NaN,15:35:00,15:35:00,StopPoint:OCEINTERCITES-87581009,9,NaN,1,0,NaN,14:58:00
197,264,2024-07-07,1,OCESN4758F3124375:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4758,0.0,657,NaN,15:35:00,15:35:00,StopPoint:OCEINTERCITES-87581009,9,NaN,1,0,NaN,14:58:00
207,94,2024-07-09,1,OCESN4758F3218347:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4758,0.0,662,NaN,15:35:00,15:35:00,StopPoint:OCEINTERCITES-87581009,9,NaN,1,0,NaN,14:58:00
217,275,2024-07-06,1,OCESN4764F3064109:2024-06-26T00:35:36Z,FR:Line::3418b67d-a81f-4dde-b3b0-60cc96ed86f4:,4764,0.0,682,NaN,21:36:00,21:36:00,StopPoint:OCEINTERCITES-87581009,9,NaN,1,0,NaN,20:59:00
